In [1]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from ast import literal_eval
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)

In [3]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [4]:
# 데이터셋 로드
test_df = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
test_df.info()

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)
test_df["choices_len"] = test_df["choices"].apply(len)

test_dataset = Dataset.from_pandas(test_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     869 non-null    int64 
 1   id             869 non-null    object
 2   paragraph      869 non-null    object
 3   problems       869 non-null    object
 4   question_plus  44 non-null     object
dtypes: int64(1), object(4)
memory usage: 34.1+ KB


In [5]:
df["choices_len"] = df["choices"].apply(len)
df['choices_len'].value_counts(dropna=False)

choices_len
5    1239
4     792
Name: count, dtype: int64

In [8]:
df4 = df[df['choices_len'] == 4]

In [10]:
train_df, valid_df = train_test_split(
    df4,
    test_size=0.15,        
    random_state=42,
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 673
    })
    validation: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 119
    })
})

In [11]:
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype=torch.float16,
    device_map="auto",
    )

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
### 정책별 System Prompt 함수
def get_system_message(row, system_prompts, prompt_policy):
    """
    row: 하나의 데이터 행
    system_prompts: {choices_len: {version: prompt_text}}
    prompt_policy: {choices_len: version}
    """
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    return system_prompts[choices_len][version]

SYSTEM_PROMPT_4_V1 = (
    "당신은 **지식 추론(Knowledge Inference) 전문가**입니다. "
    "이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. "
    "지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 "
    "가장 타당한 선택지 하나를 고르십시오."
)

SYSTEM_PROMPT_5_V1 = (
    "당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. "
    "이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. "
    "당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\n\n"
)

# PROMPT_POLICY:
# - 이번 실험(run)에서 "어떤 system prompt 버전을 사용할지"를 결정하는 설정값
# - choices_len(4 or 5) → 사용할 prompt 버전(v1, v2, ...)
# - 실험을 바꿀 때는 이 딕셔너리만 수정

SYSTEM_PROMPTS = {
    4: {
        "v1": SYSTEM_PROMPT_4_V1,
    },
    5: {
        "v1": SYSTEM_PROMPT_5_V1,
    }
}

SYSTEM_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [13]:
### 정책별 User Prompt 함수
def get_user_message(row, user_prompts, prompt_policy):
    """
    row: 데이터 행
    user_prompts: 템플릿 저장소
    prompt_policy: 버전 정책
    """
    # 메타 데이터 확인
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    
    # 해당 버전의 템플릿 세트 가져오기 (plus, no_plus가 들어있음)
    template_set = user_prompts[choices_len][version]
    
    # 데이터 준비
    paragraph = row['paragraph']
    question = row['question']
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(row['choices'])])
    q_plus = row.get('question_plus', None)
    
    # 분기 처리 및 포맷팅 (여기가 핵심!)
    # q_plus가 존재하고, nan이 아닐 때 -> Plus 템플릿 사용
    if q_plus and str(q_plus) != 'nan':
        return template_set["plus"].format(
            paragraph=paragraph,
            question_plus=q_plus, # 여기 들어감
            question=question,
            choices=choices_str
        )
    # q_plus가 없을 때 -> No Plus 템플릿 사용
    else:
        return template_set["no_plus"].format(
            paragraph=paragraph,
            question=question,
            choices=choices_str
        )
    
# =========================
# User Prompt Templates (V1)
# =========================

# 4지선다 + <보기> 있음
USER_PROMPT_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 4지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 있음
USER_PROMPT_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

USER_PROMPTS = {
    4: {
        "v1": {
            "plus": USER_PROMPT_PLUS_4_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_V1,
        },
        # "v2": {...}
    },
    5: {
        "v1": {
            "plus": USER_PROMPT_PLUS_5_V1,
            "no_plus": USER_PROMPT_NO_PLUS_5_V1,
        },
        # "v2": {...}
    }
}

USER_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [14]:
# 공통 Assistant Prompt 함수
def get_assistant_message(row):
    """
    Assistant 메시지 생성 함수.
    Qwen3 모델의 토크나이저 템플릿이 자동으로 <think> 태그를 처리하므로,
    여기서는 순수한 정답(Label) 텍스트만 반환
    """
    return str(row['answer'])

In [15]:
def build_messages(example):
    """
    원본 example(row)로부터 학습용 chat messages를 구성한다.
    - choices_len(4/5) 및 question_plus 유무에 따라 system/user 프롬프트를 선택
    - assistant는 정답 숫자만
    - 이후 평가/추적용으로 id, label도 함께 유지
    """
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)
    asst_msg = get_assistant_message(example)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": asst_msg},
        ],
        "label": int(example["answer"]),
    }

In [16]:
def to_text(example):
    """
    messages(list[dict])를 tokenizer의 chat_template 규칙에 따라
    단일 텍스트로 직렬화한다.
    """
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,  
    )
    return {"text": text}

In [17]:
def tokenize_fn(example, truncation=True, max_length=2048, padding=False):
    """
    batched=True면 example["text"]는 List[str]
    batched=False면 example["text"]는 str
    """
    tok_kwargs = dict(truncation=truncation, padding=padding)
    if truncation is True:
        tok_kwargs["max_length"] = max_length

    out = tokenizer(example["text"], **tok_kwargs)
    
    return {
        "input_ids": out["input_ids"],
        "attention_mask": out["attention_mask"],
    }

In [39]:
orig_cols = dataset["train"].column_names
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    fn_kwargs={"truncation": True, "max_length": 2048, "padding": False},
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    keep_in_memory=True,
    desc="Tokenizing",
)

Build messages:   0%|          | 0/673 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/119 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/673 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/673 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/119 [00:00<?, ? examples/s]

In [40]:
print("\n=== 변환 완료 ===")
print("Train 개수:", len(tokenized_dataset["train"]))
print("첫 번째 샘플 Keys:", tokenized_dataset["train"][0].keys())


=== 변환 완료 ===
Train 개수: 673
첫 번째 샘플 Keys: dict_keys(['id', 'label', 'input_ids', 'attention_mask'])


In [41]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 673
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 119
    })
})

In [42]:
response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

In [43]:
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def preprocess_logits_for_metrics(logits, labels, pos_from_tail=4):
    """
    반환: (batch, 5)  -> '1'~'5'에 해당하는 logits만 뽑아서 metrics 단계로 전달
    """
    # Trainer가 (logits, ...) 튜플을 줄 때가 있어서 정리
    if isinstance(logits, tuple):
        logits = logits[0]  # (B, L, V)

    # labels: (B, L), pad/무시 영역은 -100일 가능성이 큼
    # real_len = 마지막으로 labels != -100 인 위치 + 1 로 복원
    labels_t = torch.as_tensor(labels)
    not_ignored = (labels_t != -100)

    # 샘플별로 마지막 not_ignored 위치 찾기
    # (뒤에서부터 True 찾기)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)          # (B,)
    has_any = not_ignored.any(dim=1)                             # (B,)
    # real_len = seq_len - last_true_from_end
    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end

    # 만약 labels가 전부 -100인 샘플이 있으면(비정상) 그냥 seq_len로 처리
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos = (real_len - pos_from_tail).clamp(min=0, max=seq_len-1) # (B,)

    # (B, V)로 해당 위치의 logits만 gather
    logits_t = torch.as_tensor(logits)                           # (B, L, V)
    batch_idx = torch.arange(logits_t.size(0), device=logits_t.device)
    picked = logits_t[batch_idx, pos, :]                         # (B, V)

    # digit ids만 슬라이스 -> (B, 5)
    picked_digits = picked[:, DIGIT_IDS]
    return picked_digits

In [44]:
def compute_metrics(eval_pred, label_pos_from_tail=3):
    """
    eval_pred:
      - (predictions, label_ids) 튜플 형태가 가장 흔함
      - predictions: preprocess_logits_for_metrics가 반환한 (B, 5)
      - label_ids: (B, L) with -100 ignored
    반환: {"accuracy": ..., "macro_f1": ...}
    """
    if hasattr(eval_pred, "predictions"):
        preds, labels = eval_pred.predictions, eval_pred.label_ids
    else:
        preds, labels = eval_pred

    preds_t = torch.as_tensor(preds)
    pred_cls = torch.argmax(preds_t, dim=-1).cpu().numpy().astype(np.int64)  # (B,)

    labels_t = torch.as_tensor(labels)

    not_ignored = (labels_t != -100)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)
    has_any = not_ignored.any(dim=1)

    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos_label = (real_len - label_pos_from_tail).clamp(min=0, max=seq_len - 1)
    batch_idx = torch.arange(labels_t.size(0), device=labels_t.device)
    gold_tok = labels_t[batch_idx, pos_label].cpu().numpy().astype(np.int64) 

    gold_cls = gold_tok - DIGIT_IDS[0]  

    valid = (gold_cls >= 0) & (gold_cls < 5)
    pred_cls = pred_cls[valid]
    gold_cls = gold_cls[valid]

    acc = (pred_cls == gold_cls).mean() if len(gold_cls) > 0 else 0.0

    f1s = []
    for c in range(5):
        tp = np.sum((pred_cls == c) & (gold_cls == c))
        fp = np.sum((pred_cls == c) & (gold_cls != c))
        fn = np.sum((pred_cls != c) & (gold_cls == c))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        f1s.append(f1)

    macro_f1 = float(np.mean(f1s)) if len(f1s) > 0 else 0.0

    return {"accuracy": float(acc), "macro_f1": macro_f1}

In [45]:
train_dataset = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["id", "label"])

In [54]:
# 1. 모델 이름 및 양자화 설정
MODEL_NAME = "Qwen/Qwen3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# 2. 토크나이저 먼저 로드 및 PAD 설정 (중요!)
# 모델 로드 전에 PAD 토큰을 확정지어야 나중에 모델 설정에 바로 반영됩니다.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. 모델 로드 (양자화 및 장치 할당)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

# 4. 모델 설정 업데이트
model.config.use_cache = False
model.gradient_checkpointing_enable()

# 토크나이저의 PAD 설정을 모델에 동기화
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# 확인 출력
print(f"Final Pad Token: {tokenizer.pad_token} ({tokenizer.pad_token_id})")
print(f"Use Cache: {model.config.use_cache}, Grad Ckpt: {model.is_gradient_checkpointing}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Final Pad Token: <|endoftext|> (151643)
Use Cache: False, Grad Ckpt: True


In [55]:
model = prepare_model_for_kbit_training(model)

# Attention proj만
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Attention + MLP까지 (성능 더 노리되 trainable 조금 증가)
# "gate_proj", "up_proj", "down_proj" -> FFN
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# target_modules = ["q_proj", "k_proj"]

target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [ ]:
training_args = SFTConfig(
    output_dir="../../qwen-sft-results",
    
    num_train_epochs=2,
    max_seq_length=2048,
    packing=False,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    fp16=True,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # 전략 설정
    eval_strategy="steps",       # 주석 해제 (활성화)
    eval_steps=20,
    save_strategy="no",          # 저장 안 함 (비교용)
    load_best_model_at_end=False, # 저장 안 할 때는 False가 안전
    
    # 지표 및 로깅
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_steps=10,
    report_to="none",
)

In [57]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args=training_args,
)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [68]:
import torch
import gc

# 기존 객체 삭제
if 'trainer' in locals(): del trainer
if 'model' in locals(): del model

# 가비지 컬렉션 및 CUDA 캐시 비우기
gc.collect()
torch.cuda.empty_cache()

In [58]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,Accuracy,Macro F1
20,0.570600,0.122589,0.714286,0.559377
40,0.119700,0.111150,0.689076,0.540536
60,0.127000,0.108635,0.722689,0.565741
80,0.141700,0.111214,0.663866,0.525989
100,0.065300,0.123531,0.672269,0.530332
120,0.038800,0.120189,0.689076,0.542736
140,0.059300,0.112962,0.672269,0.531787
160,0.073900,0.112050,0.663866,0.524727


TrainOutput(global_step=170, training_loss=0.3064488097148783, metrics={'train_runtime': 1406.8652, 'train_samples_per_second': 0.957, 'train_steps_per_second': 0.121, 'total_flos': 4.16753022432768e+16, 'train_loss': 0.3064488097148783, 'epoch': 2.0})

### inference

In [59]:
infer_results = [] 

model.eval()
with torch.inference_mode():
    # test_ds_text 대신 eval_dataset(validation set) 사용
    for i in tqdm(range(len(eval_dataset)), desc="Validating..."):
        # 1. ID와 정답(Ground Truth) 가져오기 (데이터셋 구조에 따라 확인 필요)
        # 보통 tokenized_dataset["validation"]에 'id'와 'label'이 남아있다면 그걸 사용합니다.
        _id = tokenized_dataset["validation"][i].get("id", i)
        gt_answer = tokenized_dataset["validation"][i].get("label", "unknown") 
        
        # 2. 입력 데이터 준비 (이미 토크나이즈된 input_ids 사용)
        input_ids = torch.tensor(eval_dataset[i]['input_ids']).unsqueeze(0).to("cuda")
        attention_mask = torch.tensor(eval_dataset[i]['attention_mask']).unsqueeze(0).to("cuda")

        # 3. 모델 생성
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        # 4. 생성된 부분만 잘라내기
        input_len = input_ids.shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        # 5. 숫자 추출 로직 (뒤에서부터 찾기)
        pred = '1'
        found = False
        for char in reversed(gen_text):
            if char in ['1', '2', '3', '4', '5']:
                pred = char
                found = True
                break
        
        infer_results.append({
            "id": _id,
            "ground_truth": gt_answer, # 정답과 비교하기 위해 추가
            "prediction": pred,
            "raw_output": gen_text,
            "is_found": found
        })

df_val_results = pd.DataFrame(infer_results)

Validating...: 100%|██████████| 119/119 [01:08<00:00,  1.74it/s]


In [60]:
# 숫자를 못 찾은 비율 확인
fail_count = len([res for res in infer_results if not res['is_found']])
if fail_count > 0:
    print(f"⚠️ 경고: {fail_count}개의 데이터에서 숫자를 추출하지 못했습니다. 'raw_output'을 확인하세요.")

df_val_results[['id', 'prediction']].to_csv("val_results_no_RAG.csv", index=False)

⚠️ 경고: 30개의 데이터에서 숫자를 추출하지 못했습니다. 'raw_output'을 확인하세요.


In [61]:
infer_results

[{'id': 'generation-for-nlp-1261',
  'ground_truth': 4,
  'prediction': '1',
  'raw_output': '',
  'is_found': False},
 {'id': 'generation-for-nlp-1063',
  'ground_truth': 1,
  'prediction': '4',
  'raw_output': '4',
  'is_found': True},
 {'id': 'generation-for-nlp-1149',
  'ground_truth': 4,
  'prediction': '1',
  'raw_output': '',
  'is_found': False},
 {'id': 'generation-for-nlp-1160',
  'ground_truth': 3,
  'prediction': '3',
  'raw_output': '</think>\n\n3',
  'is_found': True},
 {'id': 'generation-for-nlp-566',
  'ground_truth': 4,
  'prediction': '4',
  'raw_output': '</think>\n\n4',
  'is_found': True},
 {'id': 'generation-for-nlp-501',
  'ground_truth': 3,
  'prediction': '4',
  'raw_output': '</think>\n\n4',
  'is_found': True},
 {'id': 'generation-for-nlp-767',
  'ground_truth': 1,
  'prediction': '1',
  'raw_output': '',
  'is_found': False},
 {'id': 'generation-for-nlp-1048',
  'ground_truth': 3,
  'prediction': '4',
  'raw_output': '</think>\n\n4',
  'is_found': True},
 {'

In [64]:
df_val_results

,id,ground_truth,prediction,raw_output,is_found
0,generation-for-nlp-1261,4,1,,False
1,generation-for-nlp-1063,1,4,4,True
2,generation-for-nlp-1149,4,1,,False
3,generation-for-nlp-1160,3,3,</think>\n\n3,True
4,generation-for-nlp-566,4,4,</think>\n\n4,True
...,...,...,...,...,...
114,generation-for-nlp-683,2,1,,False
115,generation-for-nlp-1142,4,1,,False
116,generation-for-nlp-831,4,1,,False
117,generation-for-nlp-459,1,2,2,True


In [65]:
infer_results2 = [] 

model.eval()
with torch.inference_mode():
    # test_ds_text 대신 eval_dataset(validation set) 사용
    for i in tqdm(range(len(eval_dataset)), desc="Validating..."):
        # 1. ID와 정답(Ground Truth) 가져오기 (데이터셋 구조에 따라 확인 필요)
        # 보통 tokenized_dataset["validation"]에 'id'와 'label'이 남아있다면 그걸 사용합니다.
        _id = tokenized_dataset["validation"][i].get("id", i)
        gt_answer = tokenized_dataset["validation"][i].get("label", "unknown") 
        
        # 2. 입력 데이터 준비 (이미 토크나이즈된 input_ids 사용)
        input_ids = torch.tensor(eval_dataset[i]['input_ids']).unsqueeze(0).to("cuda")
        attention_mask = torch.tensor(eval_dataset[i]['attention_mask']).unsqueeze(0).to("cuda")

        # 3. 모델 생성
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        # 4. 생성된 부분만 잘라내기
        input_len = input_ids.shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        # 5. 숫자 추출 로직 (뒤에서부터 찾기)
        pred = 'N/A' # 기본값을 숫자가 아닌 것으로 변경 (구분용)
        found = False
        
        for char in reversed(gen_text):
            if char in ['1', '2', '3', '4', '5']:
                pred = char
                found = True
                break
        
        infer_results2.append({
            "id": _id,
            "ground_truth": gt_answer,
            "prediction": pred,
            "is_found": found,
            "raw_output": repr(gen_text) # repr()을 쓰면 줄바꿈(\n) 같은 특수기호가 다 보입니다!
        })

df_val_results_2 = pd.DataFrame(infer_results2)

Validating...: 100%|██████████| 119/119 [01:07<00:00,  1.76it/s]


In [67]:
df_val_results_2[:50]

,id,ground_truth,prediction,is_found,raw_output
0,generation-for-nlp-1261,4,N/A,False,''
1,generation-for-nlp-1063,1,4,True,'4'
2,generation-for-nlp-1149,4,N/A,False,''
3,generation-for-nlp-1160,3,3,True,'</think>\n\n3'
4,generation-for-nlp-566,4,4,True,'</think>\n\n4'
5,generation-for-nlp-501,3,4,True,'</think>\n\n4'
6,generation-for-nlp-767,1,N/A,False,''
7,generation-for-nlp-1048,3,4,True,'</think>\n\n4'
8,generation-for-nlp-585,2,N/A,False,''
9,generation-for-nlp-1137,2,N/A,False,''


In [ ]:
eval_dataset